## 재사용 가능한 try/finally 동작을 원한다면 contextlib과 with문을 사용해라
- with문은 코드가 특별한 context안에서 실행되는 경우를 표현한다.
- contextlib 내장 모듈을 사용하면 객체나 함수를 with문에서 쉽게 쓸 수 있다.
- contextlib 모듈은 with 문에 쓸 수 있는 함수를 간단히 만들수 있는 contextmanager 데코레이터를 제공한다.

In [10]:
import logging
from contextlib import contextmanager

def my_function():
    logging.debug('디버깅 데이터')
    logging.error('에러 로그')
    logging.debug('추가 디버깅 데이터')

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    logger.setLevel(level)
    try:
        yield
    finally:
        # with문이 끝날떄 실행되는 블록
        logger.setLevel(logger.getEffectiveLevel())

with debug_logging(logging.DEBUG):
    print('내부')
    my_function()


print('외부')
my_function() #기본 수준은 warning이기 때문에 error 로그만 출력 된다.

DEBUG:root:디버깅 데이터
ERROR:root:에러 로그
DEBUG:root:추가 디버깅 데이터
DEBUG:root:디버깅 데이터
ERROR:root:에러 로그
DEBUG:root:추가 디버깅 데이터


내부
닫음
외부


#### with와 대상 변수 함꼐 사용하기
- with 문에 전달된 컨텍스트 매니저가 객체를 반환할 수도 있다.
- 이렇게 반환된 객체는 with 복합문의 일부로 지정된 지역변수에 대입 된다.
```py
with open('my.txt', 'w') as handle:
    handle.write('데이터 입니다.') 
```
- open을 전달 하면 as문을 통해 지정된 변수에게 파일 핸들을 전달 하고 with문을 빠져 나가면 핸들을 닫는다.
- 이러한 방식이 매변 수동으로 열고 닫는 것보다 더 파이썬 다운 방식이다.
- with문의 변수를 변경하여 로그의 이름과 레벨을 변경하여 with문 내에서 사용할 수 있다.
- logger level을 변경 하여 출력이 가능하다.


In [22]:
import logging
from contextlib import contextmanager

@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)

    try:
        yield logger
    finally:
        logger.setLevel(old_level)

with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug(f"대상: {logger.name}!")
    
logging.debug('이 메시지는 출력되지 않는다.')
    

DEBUG:my-log:대상: my-log!
DEBUG:root:이 메시지는 출력되지 않는다.
